In [1]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

c:\Users\Marcelo\Documents\Code\tcc-sentiment-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 📁 Caminhos
caminho_raw = r"C:\Users\Marcelo\Documents\Code\tcc-sentiment-analysis\data\raw"
caminho_saida = r"C:\Users\Marcelo\Documents\Code\tcc-sentiment-analysis\data\inferred"

# 🔧 Garante que o diretório de saída exista
os.makedirs(caminho_saida, exist_ok=True)

In [3]:
# Carregar modelo FinBERT
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

Device set to use cpu


In [4]:
# Listar arquivos CSV
arquivos = [f for f in os.listdir(caminho_raw) if f.endswith(".csv")]
arquivos

['noticias_AAPL.csv',
 'noticias_GOOGL.csv',
 'noticias_MSFT.csv',
 'noticias_TSLA.csv']

In [5]:
# Processar cada arquivo
for nome_arquivo in arquivos:
    caminho_csv = os.path.join(caminho_raw, nome_arquivo)
    print(f"Processando: {caminho_csv}")
    
    # Carregar o DataFrame
    df = pd.read_csv(caminho_csv)

    text_column = "Título"
    
    if text_column not in df.columns:
        print(f"Aviso: arquivo '{nome_arquivo}' não possui coluna {text_column}. Pulando.")
        continue

    # Fazer inferência de sentimento
    sentimentos = []
    for titulo in tqdm(df[text_column].astype(str), desc="Classificando"):
        try:
            resultado = sentiment_pipeline(titulo)[0]
            sentimentos.append(resultado["label"])  # 'POSITIVE', 'NEGATIVE', 'NEUTRAL'
        except Exception as e:
            print(f"Erro ao processar linha: {titulo} -> {e}")
            sentimentos.append("ERRO")
    
    # Adicionar ao DataFrame
    df["Sentimento"] = sentimentos
    
    # Salvar no diretório de destino
    caminho_arquivo_saida = os.path.join(caminho_saida, nome_arquivo)
    df.to_csv(caminho_arquivo_saida, index=False, encoding="utf-8-sig")
    print(f"Salvo em: {caminho_arquivo_saida}")

Processando: C:\Users\Marcelo\Documents\Code\tcc-sentiment-analysis\data\raw\noticias_AAPL.csv


Classificando: 100%|██████████| 228/228 [00:09<00:00, 23.44it/s]


Salvo em: C:\Users\Marcelo\Documents\Code\tcc-sentiment-analysis\data\inferred\noticias_AAPL.csv
Processando: C:\Users\Marcelo\Documents\Code\tcc-sentiment-analysis\data\raw\noticias_GOOGL.csv


Classificando: 100%|██████████| 229/229 [00:09<00:00, 24.39it/s]


Salvo em: C:\Users\Marcelo\Documents\Code\tcc-sentiment-analysis\data\inferred\noticias_GOOGL.csv
Processando: C:\Users\Marcelo\Documents\Code\tcc-sentiment-analysis\data\raw\noticias_MSFT.csv


Classificando: 100%|██████████| 231/231 [00:09<00:00, 24.36it/s]


Salvo em: C:\Users\Marcelo\Documents\Code\tcc-sentiment-analysis\data\inferred\noticias_MSFT.csv
Processando: C:\Users\Marcelo\Documents\Code\tcc-sentiment-analysis\data\raw\noticias_TSLA.csv


Classificando: 100%|██████████| 227/227 [00:09<00:00, 23.98it/s]

Salvo em: C:\Users\Marcelo\Documents\Code\tcc-sentiment-analysis\data\inferred\noticias_TSLA.csv
